In [1]:
import json
import logging
import hvplot.pandas #noqa
import polars as pl
import pandas as pd

from pydantic  import  BaseModel
from rich  import  print
from rich.console  import  Console
console=Console(record=True)

from degiro_connector.trading.api import API as TradingAPI
from degiro_connector.trading.models.credentials import build_credentials
from degiro_connector.trading.models.product_search import LookupRequest
from degiro_connector.trading.models.product_search import StocksRequest
from degiro_connector.trading.models.product_search import OptionsRequest
from degiro_connector.trading.models.product_search import LeveragedsRequest
from degiro_connector.trading.models.product_search import FuturesRequest
from degiro_connector.trading.models.product_search import FundsRequest
from degiro_connector.trading.models.product_search import ETFsRequest
from degiro_connector.trading.models.product_search import BondsRequest

with open("../Courses/PythonExercises/config/config.json") as config_file:
    config_dict=json.load(config_file)

credentials=build_credentials(override=config_dict)#credentialscanalsobebuiltfromthedictionarycontainingthedetailsfromtheconfigfile

trading_api=TradingAPI(credentials=credentials)
trading_api.connect()

'708822FA26AC6A796247C5CDFE8BF34E.prod_dcv_ch66_1'

In [ ]:
#FETCHPRODUCTS
request_stock=StocksRequest(
    #index_id=1, #AEX
    #stock_country_id=978, #NL
    exchange_id=200,#EuronextAM
    #exchange_id=212,#EuronextAMS
    #exchange_id=3001,#EuronextAMS
    #exchange_id=1006,#EuronextAMS

    offset=0,
    limit=5000,
    require_total=False,
    sort_columns="name",
    sort_types="asc",
)
product_search=trading_api.product_search(product_request=request_stock,raw=False)

products_df=pl.DataFrame(product_search.products)
print(products_df)

In [ ]:
products_df.head(1)

In [ ]:
vwdId_name=["name","symbol","vwdId"]

product_df_vwdID_noNulls=products_df.select(pl.col(vwdId_name)).drop_nulls()
print(product_df_vwdID_noNulls)

In [ ]:

concat_str_requests=["issueid:","ohlc:","price:"]

search_string = "vwdId"

try:
  # Get the index using index()
  indexvwdId = vwdId_name.index(search_string)
#   print(f"The index of '{search_string}' is: {indexvwdId}")
except ValueError:
  print(f"'{search_string}' not found in the list.")

# remember to add the "issueid:" before other things you ask

for concat_str_target in concat_str_requests:
    if concat_str_target != "issueid:" :
       concat_str_target = concat_str_target + concat_str_requests[0]
       print(concat_str_target)
    product_df_vwdID_noNulls = product_df_vwdID_noNulls.with_columns(
            (
                pl.struct(
                    vwdId_name[indexvwdId]
                ).map_batches(
                    lambda x:concat_str_target+x.struct.field(vwdId_name[indexvwdId])
                )
            ).alias(
                concat_str_target+vwdId_name[indexvwdId]
            )
        )
    
print(product_df_vwdID_noNulls)



In [ ]:
# Drop columns using list comprehension
df = product_df_vwdID_noNulls.select([col for col in product_df_vwdID_noNulls.columns if col not in vwdId_name])

print(df)



# Get the column as a list
col_list = df[:,1]

print(col_list)  # Output: [1, 2, 3]


In [ ]:
lookup_request=LookupRequest(
    search_text="APPLE",
    limit=2,
    offset=0,
    product_type_id=1,
)

product_batch=trading_api.product_search(product_request=lookup_request,raw=False)

print(product_batch)

In [ ]:
#FETCHPRODUCTS
request_stock=StocksRequest(
    #index_id=122001,#NASDAQ100
    exchange_id=663,#NASDAQ
    # You can either use `index_id` or `exchangeid`
    # See which one to use in the `ProductsConfig` table
    #is_in_us_green_list=True,#type:ignore
    #stock_country_id=846,#US
    offset=0,
    limit=100,
    require_total=True,
    sort_columns="name",
    sort_types="asc",
)
product_search=trading_api.product_search(product_request=request_stock,raw=False)

products_df=pl.DataFrame(product_search.products)
print(products_df)

In [ ]:
#Sampledata
data={"col1":["apple","banana","cherry"],
      "col2":["fuji","platano","amarena"]}
df=pl.DataFrame(data)
print(df)
#Stringtoconcatenate
concat_str="(fruit)"

##Concatenatethestringtoallelementsin"col1"
#df=df.with_columns(
#pl.col("col1").str.concat(pl.Expr(concat_str))#,name="col1_with_suffix"
#)

#df=df.groupby("col1").agg(
#pl.col("col2").str.concat("Type")
#)
#print(df)

#df=df.select(
#pl.col("col1").ma.str.concat(concat_str).alias("fruit1"),
#pl.col("col2").str.concat(concat_str).alias("fruit2"),
#)
#print(df)
selected_column="col1"

df=df.with_columns(
    (
        pl.struct([selected_column]).map_batches(
            lambda x: concat_str+x.struct.field(selected_column)
        )
    ).alias("col3")
)
print(df)

In [ ]:
product_df_vwdID_noNulls_requests=pl.concat(df_list)
print(product_df_vwdID_noNulls_requests)

In [ ]:

pl.concat(df_list)

#product_df_vwdID_SearchRequests=product_df_vwdID_noNulls.with_columns(
#pl.col(vwdId_name).str.cat(pl.Expr(concat_str_issueid)),name=concat_str_issueid+vwdId_name
#)


#importpolarsaspl

##Sampledata
#data={"col1":["apple","banana","cherry"]}
#df=pl.DataFrame(data)

##Stringtoconcatenate
#concat_str="(fruit)"

##Concatenatethestringtoallelementsin"col1"
#df=df.with_column(
#pl.col("col1").str.cat(pl.Expr(concat_str)),name="col1_with_suffix"
#)

#print(df)



In [ ]:
product_request=OptionsRequest(
input_aggregate_types='',
input_aggregate_values='',
option_exchange_id=3,
underlying_isin='FR0003500008',
search_text='',

offset=100,
limit=1000,
require_total=True,
sort_columns='expirationDate,strike',
sort_types='asc,asc',
)
print(product_request)
#FETCHDATA
option_list=trading_api.product_search(product_request=product_request,raw=False)
print(option_list)
OptionDF=pl.DataFrame(option_list.products)
print(OptionDF)

In [ ]:
#SETUPREQUEST
product_request=FuturesRequest(
future_exchange_id=1,
underlying_isin='FR0003500008',

search_text='',
offset=15,
limit=100,
require_total=True,
sort_columns='name',
sort_types='asc',
)
print(product_request)
#FETCHDATA
futures_list=trading_api.product_search(product_request=product_request)
print(futures_list)
futuresDF=pl.DataFrame(futures_list.products)

print(futuresDF)

In [ ]:
#SETUPREQUEST
product_request=LeveragedsRequest(
popular_only=False,
input_aggregate_types='',
input_aggregate_values='',

search_text='',
offset=0,
limit=5,
require_total=True,
sort_columns='name',
sort_types='asc',
)
print(product_request)
#FETCHDATA
leveraged_list=trading_api.product_search(product_request=product_request)
print(leveraged_list)
leveragedsDF=pl.DataFrame(leveraged_list.products)

print(leveragedsDF)

In [ ]:
#SETUPREQUEST
product_request=FundsRequest(
search_text='',
offset=0,
limit=5,
require_total=True,
sort_columns='name',
sort_types='asc',
)
print(product_request)
#FETCHDATA
fund_list=trading_api.product_search(product_request=product_request)
print(fund_list)
fundDF=pl.DataFrame(fund_list.products)

print(fundDF)

In [ ]:
#SETUPREQUEST
product_request=ETFsRequest(
popular_only=False,
input_aggregate_types='',
input_aggregate_values='',

search_text='',
offset=0,
limit=5,
require_total=True,
sort_columns='name',
sort_types='asc',
)
print(product_request)
#FETCHDATA
etf_list=trading_api.product_search(product_request=product_request)
etfDF=pl.DataFrame(etf_list.products)
print(etfDF)

In [ ]:
#SETUPREQUEST
product_request=BondsRequest(
bond_issuer_type_id=0,
bond_exchange_id=710,

search_text='',
offset=0,
limit=100,
require_total=True,
sort_columns='name',
sort_types='asc',
)
print(product_request)
#FETCHDATA
bond_list=trading_api.product_search(product_request=product_request)
bondDF=pl.DataFrame(bond_list.products)
print(bondDF)

In [ ]:
# df_list=[]

# for concat_str_target in concat_str_requests:
#     df_list.append(
#         product_df_vwdID_noNulls.with_columns(
#             (
#                 pl.struct(
#                     vwdId_name[indexvwdId]
#                 ).map_batches(
#                     lambda x:concat_str_target+x.struct.field(vwdId_name[indexvwdId])
#                 )
#             ).alias(
#                 concat_str_target+vwdId_name[indexvwdId]
#             )
#         )
#     )
# print(df_list)
# deTEst = product_df_vwdID_noNulls.with_columns(
#     (
#         pl.struct(
#             vwdId_name[1]
#         ).map_batches(
#             lambda x:concat_str_target+x.struct.field(vwdId_name[1])
#         )
#     ).alias(
#         concat_str_target+vwdId_name[1]
#     )
# )
# print(deTEst)